In [1]:
#imports
import numpy        as np
from scipy.optimize import brentq
from scipy.stats    import norm

#self-written shortcuts
from Methods_Core   import *


In [ ]:
def black_scholes_d1(underlying_price, strike, volatility, 
                     time_to_expiry, interest_rate, time_to_payment, forward=False):
    
    #   underlying_price: current stock price (spot price)
    #   strike: strike price of the option
    #   volatility: volatility (annual standard deviation)
    #   time_to_expiry : time to option expiry
    #   interest_rate: risk-free interest rate (continuously compounded)
    #   time_to_payment : time to expiration in years

    
    numerator1  = np.log(underlying_price / strike)
    numerator2  = interest_rate * time_to_payment * (0 if forward else 1)
    numerator3  = volatility * volatility * time_to_expiry / 2
    denominator = volatility * np.sqrt(time_to_expiry)
    
    d1          = (numerator1 + numerator2 + numerator3) / denominator  
    
    return d1
      

In [1]:
def black_scholes(option_type, underlying_price, strike, volatility, 
                  time_to_expiry, interest_rate, time_to_payment, forward=False):

    #   option_type: "C" or "P" (type of option)
    #   underlying_price: current stock price (spot price)
    #   strike: strike price of the option
    #   volatility: volatility (annual standard deviation)
    #   time_to_expiry : time to option expiry
    #   interest_rate: risk-free interest rate (continuously compounded)
    #   time_to_payment : time to expiration in years

    # Calculate discount factor
    discount_factor = calcDiscountFactor(interest_rate, time_to_payment)
    
    # Calculate d1 and d2
    d1 = black_scholes_d1(underlying_price, strike, volatility, 
                          time_to_expiry, interest_rate, time_to_payment, forward)
    d2 = d1 - volatility * np.sqrt(time_to_expiry)

    # Choose put or call
    option_type = option_type[0].upper()
        
    # Calculate call or put option price   
    if option_type == "C":  # Call option price        
        option_price = (underlying_price * norm.cdf(d1)   
                        - (1 if forward else discount_factor) * strike * norm.cdf(d2))
    elif option_type == "P": # Put option price       
        option_price = ((1 if forward else discount_factor) * strike * norm.cdf(-d2)
                        - underlying_price * norm.cdf(-d1))
    else:  # Print error for invalid option type       
        print(f"Error: Invalid option type '{option_type}'. Use 'C' for Call or 'P' for Put.")
        option_price = None  # Return None for invalid types
  
    if forward:
        option_price = option_price * discount_factor
    
    return option_price


In [ ]:
def black_scholes_vega(underlying_price, strike, volatility, 
                       time_to_expiry, interest_rate, time_to_payment, forward=False):

    #   underlying_price: current stock price (spot price)
    #   strike: strike price of the option
    #   time_to_expiry : time to option expiry
    #   interest_rate: risk-free interest rate (continuously compounded)
    #   time_to_payment : time to expiration in years


    d1 = black_scholes_d1(underlying_price, strike, volatility, 
                          time_to_expiry, interest_rate, time_to_payment, forward)
    vega_value = underlying_price * np.sqrt(time_to_expiry) * norm.pdf(d1)  
    # norm.pdf is the PDF of the standard normal distribution
    
    return vega_value


In [ ]:
def black_scholes_implied_volatility(option_price, option_type, underlying_price, strike, 
                       time_to_expiry, interest_rate, time_to_payment, forward=False, 
                       initial_guess=0.5, tol=1e-6, max_iter=100)
:

#    Solve for the implied volatility using Newton-Raphson method.

#    Parameters:
#        option_price : The observed market option price.
#        S : Spot price of the underlying asset.
#        K : Strike price of the option.
#        T : Time to expiration in years.
#        r : Risk-free interest rate.
#        initial_guess : The initial guess for volatility (default 0.2).
#        tol : The tolerance for stopping criteria (default 1e-6).
#        max_iter : Maximum number of iterations (default 100).
        
#    Returns:
#        The implied volatility.
 
    volatility = initial_guess
    for i in range(max_iter):
        
        # Calculate the option price using the current guess for volatility
        price = black_scholes(option_type, underlying_price, strike, volatility, 
                              time_to_expiry, interest_rate, time_to_payment, forward)

        vega_value = black_scholes_vega(underlying_price, strike, volatility, 
                                        time_to_expiry, interest_rate, time_to_payment, forward)
        
        # Check if Vega is zero to avoid division by zero
        if vega_value == 0:
            print("Vega is zero, cannot proceed with Newton-Raphson.")
            return None
        
        # Calculate the difference (f(sigma)) and the derivative (f'(sigma))
        diff = price - option_price
        
        # Update the volatility guess using Newton-Raphson
        vol_new = volatility - (diff / vega_value)
        
        # Check for convergence
        if abs(vol_new - volatility) < tol:
            return vol_new
        
        # Update the guess
        volatility = vol_new
    
    print(f"Maximum iterations reached. Last guess for implied volatility: {volatility}")
    
    return volatility
